In [ ]:
/*
PROJECT STRUCTURE
- ASP.NET Web Forms frontend + C# backend
- FastAPI backend for ML
- SQL Server storage
*/

// ===========================================
// 1. ASPX Frontend (Default.aspx)
// ===========================================
<%@ Page Language="C#" AutoEventWireup="true" CodeBehind="Default.aspx.cs" Inherits="MLPredictionWebApp.Default" %>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
<head runat="server">
    <title>ML Prediction Dashboard</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet" />
</head>
<body>
    <form id="form1" runat="server">
        <div class="container mt-5">
            <h3>Upload CSV & Predict with ML Models</h3>
            <asp:FileUpload ID="FileUpload1" runat="server" />
            <asp:Button ID="btnUpload" runat="server" Text="Upload CSV" CssClass="btn btn-primary mb-3" OnClick="btnUpload_Click" />
            <asp:GridView ID="GridView1" runat="server" AutoGenerateSelectButton="True" OnSelectedIndexChanged="GridView1_SelectedIndexChanged" CssClass="table table-bordered"></asp:GridView>

            <asp:Button ID="btnPredict" runat="server" Text="Predict Selected Row" CssClass="btn btn-success" OnClick="btnPredict_Click" />
            <asp:Label ID="lblResult" runat="server" CssClass="d-block mt-3"></asp:Label>
        </div>
    </form>
</body>
</html>


// ===========================================
// 2. C# Backend (Default.aspx.cs)
// ===========================================
using System;
using System.Data;
using System.IO;
using System.Web.UI;
using System.Web.UI.WebControls;
using System.Data.SqlClient;
using System.Net.Http;
using System.Text;
using Newtonsoft.Json;
using System.Collections.Generic;

namespace MLPredictionWebApp
{
    public partial class Default : Page
    {
        static DataTable csvData;

        protected void btnUpload_Click(object sender, EventArgs e)
        {
            if (FileUpload1.HasFile)
            {
                string filePath = Server.MapPath("~/App_Data/") + Path.GetFileName(FileUpload1.FileName);
                FileUpload1.SaveAs(filePath);
                csvData = LoadCsv(filePath);
                GridView1.DataSource = csvData;
                GridView1.DataBind();
            }
        }

        private DataTable LoadCsv(string csvPath)
        {
            DataTable dt = new DataTable();
            string[] lines = File.ReadAllLines(csvPath);
            string[] headers = lines[0].Split(',');
            foreach (string header in headers)
                dt.Columns.Add(header);
            for (int i = 1; i < lines.Length; i++)
                dt.Rows.Add(lines[i].Split(','));
            return dt;
        }

        protected void GridView1_SelectedIndexChanged(object sender, EventArgs e)
        {
            lblResult.Text = "Selected Row Index: " + GridView1.SelectedIndex.ToString();
        }

        protected async void btnPredict_Click(object sender, EventArgs e)
        {
            if (GridView1.SelectedIndex < 0 || csvData == null) return;
            var row = csvData.Rows[GridView1.SelectedIndex];
            List<string> rowData = new List<string>();
            List<string> columnNames = new List<string>();
            foreach (DataColumn col in csvData.Columns)
            {
                columnNames.Add(col.ColumnName);
                rowData.Add(row[col].ToString());
            }

            var input = new
            {
                row_data = rowData,
                column_names = columnNames
            };

            string json = JsonConvert.SerializeObject(input);

            using (HttpClient client = new HttpClient())
            {
                var content = new StringContent(json, Encoding.UTF8, "application/json");
                var response = await client.PostAsync("http://127.0.0.1:8000/predict", content);
                string result = await response.Content.ReadAsStringAsync();
                lblResult.Text = result;
                SaveToDatabase(json, result);
            }
        }

        private void SaveToDatabase(string input, string prediction)
        {
            using (SqlConnection conn = new SqlConnection("Data Source=localhost;Initial Catalog=MLDB;Integrated Security=True"))
            {
                conn.Open();
                SqlCommand cmd = new SqlCommand("INSERT INTO Predictions(InputData, PredictionData) VALUES(@input, @prediction)", conn);
                cmd.Parameters.AddWithValue("@input", input);
                cmd.Parameters.AddWithValue("@prediction", prediction);
                cmd.ExecuteNonQuery();
            }
        }
    }
}


// ===========================================
// 3. SQL Table (Predictions)
// ===========================================
-- Run in SQL Server
CREATE TABLE Predictions (
    ID INT PRIMARY KEY IDENTITY,
    InputData NVARCHAR(MAX),
    PredictionData NVARCHAR(MAX),
    Timestamp DATETIME DEFAULT GETDATE()
);


// ===========================================
// 4. FastAPI Backend (main.py)
// ===========================================
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import numpy as np

app = FastAPI()

# Load your 5 models
model1 = joblib.load("model1.pkl")
model2 = joblib.load("model2.pkl")
model3 = joblib.load("model3.pkl")
model4 = joblib.load("model4.pkl")
model5 = joblib.load("model5.pkl")

class RowData(BaseModel):
    row_data: list
    column_names: list

@app.post("/predict")
def predict(row: RowData):
    d = row.row_data
    cols = row.column_names
    results = {}

    try:
        def extract_features(indices):
            return [float(d[cols.index(col)]) for col in indices]

        # Model 1
        model1_inputs = ['colA', 'colB', 'colC', 'colD', 'colE']
        model1_target = 'colF'
        input1 = np.array(extract_features(model1_inputs)).reshape(1, -1)
        actual1 = float(d[cols.index(model1_target)])
        results['model1'] = {
            'input': dict(zip(model1_inputs, input1.flatten())),
            'actual': {model1_target: actual1},
            'predicted': model1.predict(input1)[0]
        }

        # Model 2
        model2_inputs = ['colA', 'colB', 'colC', 'colD', 'colE', 'colF']
        model2_target = 'colG'
        input2 = np.array(extract_features(model2_inputs)).reshape(1, -1)
        actual2 = float(d[cols.index(model2_target)])
        results['model2'] = {
            'input': dict(zip(model2_inputs, input2.flatten())),
            'actual': {model2_target: actual2},
            'predicted': model2.predict(input2)[0]
        }

        # Model 3
        model3_inputs = ['colA', 'colB', 'colC', 'colD', 'colE', 'colF', 'colH', 'colI', 'colJ']
        model3_target = 'colJ'
        input3 = np.array(extract_features(model3_inputs)).reshape(1, -1)
        actual3 = float(d[cols.index(model3_target)])
        results['model3'] = {
            'input': dict(zip(model3_inputs, input3.flatten())),
            'actual': {model3_target: actual3},
            'predicted': model3.predict(input3)[0]
        }

        # Model 4
        model4_inputs = cols[:12]
        model4_target = cols[12]
        input4 = np.array(extract_features(model4_inputs)).reshape(1, -1)
        actual4 = float(d[cols.index(model4_target)])
        results['model4'] = {
            'input': dict(zip(model4_inputs, input4.flatten())),
            'actual': {model4_target: actual4},
            'predicted': model4.predict(input4)[0]
        }

        # Model 5
        model5_inputs = cols[:25]
        model5_target = cols[25]
        input5 = np.array(extract_features(model5_inputs)).reshape(1, -1)
        actual5 = float(d[cols.index(model5_target)])
        results['model5'] = {
            'input': dict(zip(model5_inputs, input5.flatten())),
            'actual': {model5_target: actual5},
            'predicted': model5.predict(input5)[0]
        }

    except Exception as e:
        return {"error": str(e)}

    return results
